In [6]:
import pandas as pd
df_bikes = pd.read_csv('bike_rentals_cleaned.csv')

In [7]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2, shuffle=True)

# Cross validation

In [9]:
from sklearn.model_selection import cross_val_score
def cross_val(classifier, num_splits=10): 
    model = classifier 
    scores = cross_val_score(model, X, y, cv=num_splits) 
    print('Accuracy:', np.round(scores,2)) 
    print('Accuracy mean: %0.2f' % (scores.mean()))

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10, random_state=2, n_jobs=-1)

scores = cross_val_score(rf, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))


RMSE: [ 801.486  579.987  551.347  846.698  895.05  1097.522  893.738  809.284
  833.488 2145.046]
RMSE mean: 945.365


In [11]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(verbosity=0) 

In [12]:
scores = cross_val_score(xgbr, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

RMSE: [ 717.646  692.804  520.7    737.676  835.961 1006.237  991.342  747.612
  891.994 1731.128]
RMSE mean: 887.310


In [ ]:
>>> # явно требовать эту экспериментальную функцию
>>> from sklearn.experimental import enable_halving_search_cv # noqa
>>> # теперь вы можете нормально импортировать из model_selection
>>> from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
from sklearn.model_selection import HalvingGridSearchCV

In [1]:
#from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor

def randomized_search_reg(params, runs=16, reg=RandomForestRegressor(random_state=2,n_jobs=-1)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', cv=10, n_jobs=-1, random_state=2)
    rand_reg.fit(X_train, y_train)
    best_model = rand_reg.best_estimator_
    best_params = rand_reg.best_params_
    print("Best params:", best_params)
    best_score = np.sqrt(-rand_reg.best_score_)
    print("Training score: {:.3f}".format(best_score))
    y_pred = best_model.predict(X_test)

    rmse_test = MSE(y_test, y_pred)**0.5
    print('Test set score: {:.3f}'.format(rmse_test))
    
    #scores = cross_val_score(rand_reg, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)
    #rmse = np.sqrt(-scores)
    #print('RMSE:', np.round(rmse, 3))
    #print('RMSE mean: %0.3f' % (rmse.mean()))

In [55]:
import numpy as np
randomized_search_reg(params={'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075,0.01, 0.05],
                              'min_samples_split':[2, 0.01, 0.02, 0.03, 0.04, 0.06, 0.08,0.1],
                              'min_samples_leaf':[1,2,4,6,8,10,20,30],
                              'min_impurity_decrease':[0.0, 0.01, 0.05,0.10, 0.15, 0.2],
                              'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
                              'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                              'max_depth':[None,2,4,6,8,10,20]})

Best params: {'min_weight_fraction_leaf': 0.0, 'min_samples_split': 0.03, 'min_samples_leaf': 6, 'min_impurity_decrease': 0.05, 'max_leaf_nodes': 25, 'max_features': 0.7, 'max_depth': None}
Training score: 759.076
Test set score: 701.802


In [26]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor

def randomized_search_reg(params, runs=32, reg=xgb.XGBRegressor(verbosity=0)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', cv=10, n_jobs=-1, random_state=2)
    rand_reg.fit(X_train, y_train)
    best_model = rand_reg.best_estimator_
    best_params = rand_reg.best_params_
    print("Best params:", best_params)
    best_score = np.sqrt(-rand_reg.best_score_)
    print("Training score: {:.3f}".format(best_score))
    y_pred = best_model.predict(X_test)
    rmse_test = MSE(y_test, y_pred)**0.5
    print('Test set score: {:.3f}'.format(rmse_test))

import numpy as np
randomized_search_reg(params={'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075,0.01, 0.05],
                              'min_samples_split':[2, 0.01, 0.02, 0.03, 0.04, 0.06, 0.08,0.1],
                              'min_samples_leaf':[1,2,4,6,8,10,20,30],
                              'min_impurity_decrease':[0.0, 0.01, 0.05,0.10, 0.15, 0.2],
                              'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
                              'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                              'max_depth':[None,2,4,6,8,10,20],
                              'eta':[0.1,0.2,0.3,0.4,0.5],
                              'lambda':[0,1,2,3,4,5]
                             })

Best params: {'min_weight_fraction_leaf': 0.005, 'min_samples_split': 0.01, 'min_samples_leaf': 8, 'min_impurity_decrease': 0.2, 'max_leaf_nodes': 30, 'max_features': 0.6, 'max_depth': 2, 'lambda': 2, 'eta': 0.2}
Training score: 627.360
Test set score: 664.637


In [15]:
648

648

In [34]:
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

def randomized_search_reg(params, runs=1, reg=CatBoostRegressor(random_state=2)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', n_jobs=-1, random_state=2)
    rand_reg.fit(X_train, y_train)
    best_model = rand_reg.best_estimator_
    best_params = rand_reg.best_params_
    print("Best params:", best_params)
    best_score = np.sqrt(-rand_reg.best_score_)
    print("Training score: {:.3f}".format(best_score))
    y_pred = best_model.predict(X_test)

    rmse_test = MSE(y_test, y_pred)**0.5
    print('Test set score: {:.3f}'.format(rmse_test))
    
import numpy as np
randomized_search_reg(params={#'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075,0.01, 0.05],
                              #'min_samples_split':[2, 0.01, 0.02, 0.03, 0.04, 0.06, 0.08,0.1],
                              'depth':[8,10,12,14,16],
                              'l2_leaf_reg':[1,2,3,4],
                              #'min_impurity_decrease':[0.0, 0.01, 0.05,0.10, 0.15, 0.2],
                              'iterations':[10],
                              'bagging_temperature':[25,30,35,40],
                              #'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                              #'max_depth':[None,2,4,6,8,10,20]
})


KeyboardInterrupt: 

In [75]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

rng = np.random.RandomState(1)
#regr_2 = AdaBoostRegressor(CatBoostRegressor(bagging_temperature=35,depth=10,iterations=200,l2_leaf_reg=1, loss_function='RMSE'), n_estimators=10, random_state=rng)
regr_2 = CatBoostRegressor(l2_leaf_reg=1, loss_function='RMSE')
#depth=10,iterations=200, l2_leaf_reg=1, loss_function='RMSE')

parameters = {'depth' : [6],
'learning_rate' : [0.3,0.4,0.5,0.6],
'iterations' : [1000]
}
grid = RandomizedSearchCV(regr_2, parameters,n_iter=3,n_jobs=-1)
grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

0:	learn: 1526.4969080	total: 2.11ms	remaining: 2.11s
1:	learn: 1278.4033515	total: 2.96ms	remaining: 1.48s
2:	learn: 1071.2010559	total: 5.07ms	remaining: 1.68s
3:	learn: 931.5079838	total: 7.25ms	remaining: 1.81s
4:	learn: 820.2559493	total: 9.64ms	remaining: 1.92s
5:	learn: 757.3071275	total: 12ms	remaining: 1.98s
6:	learn: 709.1306949	total: 13.9ms	remaining: 1.97s
7:	learn: 670.4668695	total: 27ms	remaining: 3.35s
8:	learn: 633.8271385	total: 29.3ms	remaining: 3.22s
9:	learn: 607.8349625	total: 31.6ms	remaining: 3.13s
10:	learn: 589.3195358	total: 33.9ms	remaining: 3.05s
11:	learn: 568.0271672	total: 35.9ms	remaining: 2.95s
12:	learn: 556.1750265	total: 37.6ms	remaining: 2.85s
13:	learn: 540.7302455	total: 39.4ms	remaining: 2.77s
14:	learn: 523.4753863	total: 41.2ms	remaining: 2.71s
15:	learn: 509.6504046	total: 42.9ms	remaining: 2.64s
16:	learn: 491.2805430	total: 45.3ms	remaining: 2.62s
17:	learn: 484.5579798	total: 51.8ms	remaining: 2.82s
18:	learn: 475.9010224	total: 56.2ms	re

172:	learn: 61.1978974	total: 638ms	remaining: 3.05s
173:	learn: 60.1939931	total: 639ms	remaining: 3.04s
174:	learn: 59.6871542	total: 642ms	remaining: 3.02s
175:	learn: 59.1070701	total: 644ms	remaining: 3.01s
176:	learn: 58.6399732	total: 646ms	remaining: 3s
177:	learn: 57.5504010	total: 648ms	remaining: 2.99s
178:	learn: 56.9796546	total: 655ms	remaining: 3s
179:	learn: 56.5557227	total: 662ms	remaining: 3.01s
180:	learn: 55.8327780	total: 663ms	remaining: 3s
181:	learn: 55.4510705	total: 665ms	remaining: 2.99s
182:	learn: 55.1063577	total: 667ms	remaining: 2.98s
183:	learn: 54.9408229	total: 670ms	remaining: 2.97s
184:	learn: 54.3918684	total: 672ms	remaining: 2.96s
185:	learn: 53.8746305	total: 674ms	remaining: 2.95s
186:	learn: 53.2004099	total: 676ms	remaining: 2.94s
187:	learn: 52.5959405	total: 679ms	remaining: 2.93s
188:	learn: 51.7472797	total: 690ms	remaining: 2.96s
189:	learn: 51.0147375	total: 692ms	remaining: 2.95s
190:	learn: 50.6539961	total: 694ms	remaining: 2.94s
19

364:	learn: 11.0497158	total: 1.42s	remaining: 2.46s
365:	learn: 10.9998906	total: 1.43s	remaining: 2.48s
366:	learn: 10.9115331	total: 1.44s	remaining: 2.48s
367:	learn: 10.8050877	total: 1.44s	remaining: 2.47s
368:	learn: 10.7474071	total: 1.44s	remaining: 2.46s
369:	learn: 10.6444314	total: 1.44s	remaining: 2.46s
370:	learn: 10.5701240	total: 1.45s	remaining: 2.45s
371:	learn: 10.4071927	total: 1.46s	remaining: 2.46s
372:	learn: 10.3570250	total: 1.46s	remaining: 2.45s
373:	learn: 10.2396777	total: 1.46s	remaining: 2.45s
374:	learn: 10.1385415	total: 1.46s	remaining: 2.44s
375:	learn: 10.1028421	total: 1.47s	remaining: 2.43s
376:	learn: 9.9970721	total: 1.47s	remaining: 2.42s
377:	learn: 9.9882829	total: 1.47s	remaining: 2.42s
378:	learn: 9.9199943	total: 1.47s	remaining: 2.41s
379:	learn: 9.8584191	total: 1.47s	remaining: 2.4s
380:	learn: 9.7702077	total: 1.48s	remaining: 2.4s
381:	learn: 9.6958098	total: 1.49s	remaining: 2.41s
382:	learn: 9.5662932	total: 1.49s	remaining: 2.4s
383

558:	learn: 2.3840945	total: 2.08s	remaining: 1.64s
559:	learn: 2.3645271	total: 2.08s	remaining: 1.64s
560:	learn: 2.3573792	total: 2.08s	remaining: 1.63s
561:	learn: 2.3291806	total: 2.08s	remaining: 1.62s
562:	learn: 2.3103159	total: 2.09s	remaining: 1.62s
563:	learn: 2.2876005	total: 2.09s	remaining: 1.61s
564:	learn: 2.2560105	total: 2.09s	remaining: 1.61s
565:	learn: 2.2400205	total: 2.11s	remaining: 1.62s
566:	learn: 2.2314088	total: 2.11s	remaining: 1.61s
567:	learn: 2.2095940	total: 2.11s	remaining: 1.61s
568:	learn: 2.1941214	total: 2.11s	remaining: 1.6s
569:	learn: 2.1741574	total: 2.13s	remaining: 1.61s
570:	learn: 2.1578451	total: 2.13s	remaining: 1.6s
571:	learn: 2.1460626	total: 2.14s	remaining: 1.6s
572:	learn: 2.1221419	total: 2.14s	remaining: 1.59s
573:	learn: 2.1012186	total: 2.14s	remaining: 1.59s
574:	learn: 2.0853343	total: 2.14s	remaining: 1.58s
575:	learn: 2.0614984	total: 2.15s	remaining: 1.58s
576:	learn: 2.0487093	total: 2.16s	remaining: 1.58s
577:	learn: 2.0

758:	learn: 0.5127049	total: 2.71s	remaining: 859ms
759:	learn: 0.5108018	total: 2.71s	remaining: 855ms
760:	learn: 0.5054677	total: 2.71s	remaining: 851ms
761:	learn: 0.5046011	total: 2.71s	remaining: 847ms
762:	learn: 0.4991590	total: 2.71s	remaining: 843ms
763:	learn: 0.4936918	total: 2.72s	remaining: 840ms
764:	learn: 0.4900834	total: 2.72s	remaining: 836ms
765:	learn: 0.4877592	total: 2.72s	remaining: 832ms
766:	learn: 0.4853251	total: 2.73s	remaining: 828ms
767:	learn: 0.4831117	total: 2.73s	remaining: 824ms
768:	learn: 0.4815632	total: 2.73s	remaining: 820ms
769:	learn: 0.4753240	total: 2.73s	remaining: 816ms
770:	learn: 0.4751304	total: 2.73s	remaining: 812ms
771:	learn: 0.4700277	total: 2.73s	remaining: 808ms
772:	learn: 0.4680302	total: 2.73s	remaining: 803ms
773:	learn: 0.4650473	total: 2.74s	remaining: 799ms
774:	learn: 0.4605043	total: 2.74s	remaining: 795ms
775:	learn: 0.4577022	total: 2.74s	remaining: 791ms
776:	learn: 0.4555651	total: 2.74s	remaining: 788ms
777:	learn: 

941:	learn: 0.1442807	total: 3.21s	remaining: 198ms
942:	learn: 0.1435485	total: 3.21s	remaining: 194ms
943:	learn: 0.1425451	total: 3.21s	remaining: 191ms
944:	learn: 0.1413836	total: 3.21s	remaining: 187ms
945:	learn: 0.1402099	total: 3.22s	remaining: 184ms
946:	learn: 0.1392416	total: 3.22s	remaining: 180ms
947:	learn: 0.1384579	total: 3.22s	remaining: 177ms
948:	learn: 0.1358093	total: 3.22s	remaining: 173ms
949:	learn: 0.1349498	total: 3.23s	remaining: 170ms
950:	learn: 0.1347635	total: 3.23s	remaining: 166ms
951:	learn: 0.1329450	total: 3.23s	remaining: 163ms
952:	learn: 0.1325979	total: 3.23s	remaining: 159ms
953:	learn: 0.1319738	total: 3.23s	remaining: 156ms
954:	learn: 0.1310369	total: 3.25s	remaining: 153ms
955:	learn: 0.1298384	total: 3.25s	remaining: 150ms
956:	learn: 0.1294634	total: 3.25s	remaining: 146ms
957:	learn: 0.1286207	total: 3.25s	remaining: 143ms
958:	learn: 0.1275201	total: 3.25s	remaining: 139ms
959:	learn: 0.1263926	total: 3.26s	remaining: 136ms
960:	learn: 

In [ ]:
586.872

In [74]:
grid.best_params_

{'learning_rate': 0.3, 'iterations': 1000, 'depth': 6}

In [77]:
regr_2 = CatBoostRegressor(loss_function='RMSE', l2_leaf_reg=1)
regr_2.fit(X_train, y_train)
y_pred = regr_2.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

0:	learn: 1879.4569835	total: 1.81ms	remaining: 1.81s
1:	learn: 1841.8097528	total: 2.67ms	remaining: 1.33s
2:	learn: 1801.7388328	total: 4.21ms	remaining: 1.4s
3:	learn: 1763.9359081	total: 5.89ms	remaining: 1.47s
4:	learn: 1724.9852767	total: 7.58ms	remaining: 1.51s
5:	learn: 1690.4781673	total: 9.3ms	remaining: 1.54s
6:	learn: 1655.9520763	total: 11.1ms	remaining: 1.57s
7:	learn: 1621.3840731	total: 12.8ms	remaining: 1.58s
8:	learn: 1587.3519001	total: 20.3ms	remaining: 2.24s
9:	learn: 1555.3307291	total: 23.2ms	remaining: 2.29s
10:	learn: 1523.1461402	total: 24.8ms	remaining: 2.23s
11:	learn: 1492.2488833	total: 26.6ms	remaining: 2.19s
12:	learn: 1463.5047689	total: 28.3ms	remaining: 2.15s
13:	learn: 1434.5343650	total: 30.8ms	remaining: 2.17s
14:	learn: 1406.0892681	total: 32.5ms	remaining: 2.13s
15:	learn: 1377.4464174	total: 34.2ms	remaining: 2.1s
16:	learn: 1351.8387694	total: 35.9ms	remaining: 2.08s
17:	learn: 1325.0505141	total: 37.6ms	remaining: 2.05s
18:	learn: 1299.8601108

168:	learn: 449.9937396	total: 460ms	remaining: 2.26s
169:	learn: 448.6800294	total: 462ms	remaining: 2.25s
170:	learn: 447.7875488	total: 464ms	remaining: 2.25s
171:	learn: 446.5737200	total: 469ms	remaining: 2.26s
172:	learn: 445.6739252	total: 474ms	remaining: 2.27s
173:	learn: 444.7792091	total: 481ms	remaining: 2.28s
174:	learn: 443.7768701	total: 483ms	remaining: 2.27s
175:	learn: 442.4311467	total: 485ms	remaining: 2.27s
176:	learn: 441.6092094	total: 487ms	remaining: 2.26s
177:	learn: 440.8075016	total: 488ms	remaining: 2.25s
178:	learn: 439.9091724	total: 490ms	remaining: 2.25s
179:	learn: 438.7989095	total: 491ms	remaining: 2.24s
180:	learn: 437.9419723	total: 500ms	remaining: 2.26s
181:	learn: 436.5049121	total: 501ms	remaining: 2.25s
182:	learn: 435.8849315	total: 503ms	remaining: 2.25s
183:	learn: 435.0324449	total: 504ms	remaining: 2.24s
184:	learn: 434.3489952	total: 506ms	remaining: 2.23s
185:	learn: 433.2153223	total: 508ms	remaining: 2.22s
186:	learn: 432.5106413	tota

372:	learn: 307.9287676	total: 939ms	remaining: 1.58s
373:	learn: 307.5620912	total: 941ms	remaining: 1.57s
374:	learn: 306.9328676	total: 942ms	remaining: 1.57s
375:	learn: 306.4870175	total: 944ms	remaining: 1.57s
376:	learn: 306.1823121	total: 946ms	remaining: 1.56s
377:	learn: 305.7322165	total: 956ms	remaining: 1.57s
378:	learn: 305.4553887	total: 958ms	remaining: 1.57s
379:	learn: 305.2049330	total: 960ms	remaining: 1.57s
380:	learn: 305.0444917	total: 964ms	remaining: 1.57s
381:	learn: 304.3258114	total: 966ms	remaining: 1.56s
382:	learn: 303.9874966	total: 967ms	remaining: 1.56s
383:	learn: 303.3953004	total: 969ms	remaining: 1.55s
384:	learn: 302.9168286	total: 970ms	remaining: 1.55s
385:	learn: 302.6832193	total: 972ms	remaining: 1.55s
386:	learn: 302.1693077	total: 973ms	remaining: 1.54s
387:	learn: 301.5640670	total: 975ms	remaining: 1.54s
388:	learn: 300.5881744	total: 985ms	remaining: 1.55s
389:	learn: 300.2619563	total: 987ms	remaining: 1.54s
390:	learn: 299.8474239	tota

570:	learn: 231.9984470	total: 1.41s	remaining: 1.06s
571:	learn: 231.7672723	total: 1.42s	remaining: 1.06s
572:	learn: 231.3155373	total: 1.42s	remaining: 1.06s
573:	learn: 231.1123302	total: 1.42s	remaining: 1.05s
574:	learn: 230.8805049	total: 1.43s	remaining: 1.06s
575:	learn: 230.6545777	total: 1.43s	remaining: 1.05s
576:	learn: 230.3143564	total: 1.44s	remaining: 1.05s
577:	learn: 229.9588786	total: 1.44s	remaining: 1.05s
578:	learn: 229.4615622	total: 1.44s	remaining: 1.05s
579:	learn: 229.1088382	total: 1.44s	remaining: 1.04s
580:	learn: 228.8867262	total: 1.45s	remaining: 1.04s
581:	learn: 228.7574347	total: 1.45s	remaining: 1.04s
582:	learn: 228.4172334	total: 1.45s	remaining: 1.03s
583:	learn: 228.0330214	total: 1.45s	remaining: 1.03s
584:	learn: 227.8773805	total: 1.45s	remaining: 1.03s
585:	learn: 227.6125640	total: 1.46s	remaining: 1.03s
586:	learn: 227.1514084	total: 1.46s	remaining: 1.03s
587:	learn: 226.8453635	total: 1.46s	remaining: 1.03s
588:	learn: 226.4859883	tota

742:	learn: 184.9719385	total: 1.84s	remaining: 637ms
743:	learn: 184.8671687	total: 1.84s	remaining: 635ms
744:	learn: 184.7032383	total: 1.85s	remaining: 633ms
745:	learn: 184.4384503	total: 1.85s	remaining: 631ms
746:	learn: 184.1674511	total: 1.85s	remaining: 628ms
747:	learn: 183.8781509	total: 1.86s	remaining: 625ms
748:	learn: 183.7765618	total: 1.87s	remaining: 626ms
749:	learn: 183.5088476	total: 1.87s	remaining: 623ms
750:	learn: 183.1540338	total: 1.87s	remaining: 620ms
751:	learn: 182.9929225	total: 1.87s	remaining: 618ms
752:	learn: 182.8133822	total: 1.88s	remaining: 615ms
753:	learn: 182.5944875	total: 1.88s	remaining: 612ms
754:	learn: 182.4937030	total: 1.88s	remaining: 610ms
755:	learn: 182.2877948	total: 1.88s	remaining: 607ms
756:	learn: 182.0496857	total: 1.88s	remaining: 605ms
757:	learn: 181.9421733	total: 1.88s	remaining: 602ms
758:	learn: 181.6192191	total: 1.89s	remaining: 599ms
759:	learn: 181.3293630	total: 1.9s	remaining: 599ms
760:	learn: 181.0252136	total

902:	learn: 151.1260728	total: 2.32s	remaining: 250ms
903:	learn: 150.9447283	total: 2.33s	remaining: 247ms
904:	learn: 150.8060586	total: 2.33s	remaining: 245ms
905:	learn: 150.6328951	total: 2.33s	remaining: 242ms
906:	learn: 150.3696621	total: 2.35s	remaining: 241ms
907:	learn: 150.3175194	total: 2.36s	remaining: 239ms
908:	learn: 150.2679655	total: 2.36s	remaining: 236ms
909:	learn: 150.1778079	total: 2.36s	remaining: 234ms
910:	learn: 150.0142646	total: 2.37s	remaining: 231ms
911:	learn: 149.9153251	total: 2.37s	remaining: 228ms
912:	learn: 149.6528607	total: 2.37s	remaining: 226ms
913:	learn: 149.4964848	total: 2.37s	remaining: 223ms
914:	learn: 149.2979761	total: 2.38s	remaining: 221ms
915:	learn: 149.1891003	total: 2.38s	remaining: 218ms
916:	learn: 148.9540319	total: 2.38s	remaining: 216ms
917:	learn: 148.7763707	total: 2.38s	remaining: 213ms
918:	learn: 148.6403050	total: 2.38s	remaining: 210ms
919:	learn: 148.4157018	total: 2.39s	remaining: 208ms
920:	learn: 148.2888357	tota

In [46]:
regr_2 = CatBoostRegressor(l2_leaf_reg=1, loss_function='RMSE',depth=6,iterations = 100, learning_rate=0.1)
regr_2.fit(X_train, y_train)
y_pred = regr_2.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

0:	learn: 1784.8598521	total: 6.48ms	remaining: 641ms
1:	learn: 1669.6735828	total: 7.28ms	remaining: 357ms
2:	learn: 1554.9667664	total: 9.23ms	remaining: 299ms
3:	learn: 1457.0463795	total: 11ms	remaining: 265ms
4:	learn: 1368.2063433	total: 12.5ms	remaining: 238ms
5:	learn: 1288.5935157	total: 14.1ms	remaining: 221ms
6:	learn: 1209.6235716	total: 16.2ms	remaining: 216ms
7:	learn: 1140.2537615	total: 19.8ms	remaining: 227ms
8:	learn: 1073.6916801	total: 21.8ms	remaining: 221ms
9:	learn: 1020.5658057	total: 31.5ms	remaining: 283ms
10:	learn: 966.6202106	total: 33.7ms	remaining: 273ms
11:	learn: 919.9315141	total: 35.3ms	remaining: 259ms
12:	learn: 881.9374999	total: 36.9ms	remaining: 247ms
13:	learn: 846.7582029	total: 38.9ms	remaining: 239ms
14:	learn: 813.3769370	total: 40.6ms	remaining: 230ms
15:	learn: 785.6928075	total: 43ms	remaining: 226ms
16:	learn: 755.9083878	total: 44.9ms	remaining: 219ms
17:	learn: 741.4705707	total: 45.6ms	remaining: 208ms
18:	learn: 718.9369520	total: 47